### imports

In [3]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import re

### code

In [2]:
pdf_path = 'survey_2016.pdf'

In [4]:
reader = PdfReader(pdf_path)

In [5]:
text_by_page = []
for page_num in range(len(reader.pages)):
    page = reader.pages[page_num]
    text = page.extract_text()
    text_by_page.append(text)

In [6]:
def initial_cleanup(df):
    df = df.iloc[5:]
    df = df.iloc[:-1]
    df = df.drop(df.columns[10:], axis=1)
    return df

In [7]:
def generate_df(text): 
    lines = text.split('\n')  # Split the text into lines
    data = [line.split() for line in lines]
    
    df = pd.DataFrame(data)
    #df = initial_cleanup(df)
    #df = rename_cols(df)
    #df = shift_row_right(df)
    
    # Print the DataFrame to inspect the result
    return df

In [8]:
text0 = text_by_page[0]
text1 = text_by_page[1]
text2 = text_by_page[2]

In [9]:
text1

' 2011 North Carolina General Assembly2016 Contingent Congressional Plan CorrectedCounty District County PopDistrict      \nPopCounty \nDistrict Pop% County in \nDistrict% of District \nin CountyTotal Population by County and District\nGraham118,861733,4998,861100.0%1.208%Granville159,916733,49959,916100.0%8.169%Greene321,362733,49821,362100.0%2.912%Guilford6488,406733,498162,41833.255%22.143%Guilford13488,406733,499325,98866.745%44.443%Halifax154,691733,49954,691100.0%7.456%Harnett2114,678733,499114,678100.0%15.634%Haywood1159,036733,49959,036100.0%8.049%Henderson11106,740733,499106,740100.0%14.552%Hertford124,669733,49924,669100.0%3.363%Hoke846,952733,49946,952100.0%6.401%Hyde35,810733,4985,810100.0%0.792%Iredell10159,437733,4996,0423.79%0.824%Iredell13159,437733,499153,39596.21%20.913%Jackson1140,271733,49940,271100.0%5.49%Johnston2168,878733,499109,33264.74%14.906%Johnston7168,878733,49959,54635.26%8.118%Jones310,153733,49810,153100.0%1.384%Lee657,866733,49857,866100.0%7.889%Lenoir

In [12]:
def generate_df1(text1):
    # Regular expression to capture the needed data
    pattern = r'([A-Za-z ]+)(\d{1,3}(?:,\d{3})*)(\d{1,3}(?:,\d{3})*)(\d{1,3}(?:,\d{3})*)(\d+\.\d+)%(\d+\.\d+)%'
    
    # Find all matches using regex
    matches = re.findall(pattern, text1)
    
    # Create a DataFrame from the matches
    df = pd.DataFrame(matches, columns=["County", "County Pop", "District Pop", "County District Pop", "Percent County in District", "Percent District in County"])
    
    # Clean up numeric columns by removing commas and converting to integers or floats
    df["County Pop"] = df["County Pop"].str.replace(",", "").astype(int)
    df["District Pop"] = df["District Pop"].str.replace(",", "").astype(int)
    df["County District Pop"] = df["County District Pop"].str.replace(",", "").astype(int)
    df["Percent County in District"] = df["Percent County in District"].astype(float)
    df["Percent District in County"] = df["Percent District in County"].astype(float)

    
    # Display the resulting DataFrame
    return df

In [13]:
def process_county_pop_and_district(row):
    county_pop_str = str(row["County Pop"])
    county_district_pop_str = str(row["County District Pop"])
    
    # The district is the part of the County Pop that is not in the County District Pop
    district = int(county_pop_str[:len(county_pop_str) - len(county_district_pop_str)])
    
    # Remove the district part from County Pop
    updated_county_pop_str = county_pop_str[len(str(district)):]  # Remove the district part
    updated_county_pop = int(updated_county_pop_str)
    
    return district, updated_county_pop

In [14]:
#df0 = generate_text0(text0)
df1 = generate_df1(text1)
#df2 = generate_text2(text2)

In [15]:
df2 = generate_df1(text2)

In [16]:
# Apply the combined function to the DataFrame
df1["District"], df1["County Pop"] = zip(*df1.apply(process_county_pop_and_district, axis=1))

In [17]:
# Apply the combined function to the DataFrame
df2["District"], df2["County Pop"] = zip(*df2.apply(process_county_pop_and_district, axis=1))

In [18]:
def generate0(text):
    # Regular expression to match the required data
    pattern = r"([A-Za-z ]+)\s+(\d+)\s+([\d,]+)\s+733,499\s+([\d,]+)\s+(\d+\.\d+)%\s+(\d+\.\d+)%"
    
    # Find all matches
    matches = re.findall(pattern, text)
    
    # List to store structured data
    data = []
    
    # Process matches and store them in the list
    for match in matches:
        county = match[0].strip()
        district = match[1]
        county_pop = match[2].replace(",", "")  # Remove commas
        district_pop = "733499"  # Fixed district population value
        county_district_pop = match[3].replace(",", "")  # Remove commas
        percent_county_in_district = match[4]
        percent_of_district_in_county = match[5]
        
        # Append the row to the data list
        data.append([county, district, county_pop, district_pop, county_district_pop, percent_county_in_district, percent_of_district_in_county])
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=["County", "District", "County Pop", "District Pop", "County District Pop", "Percent County in District", "Percent of District in County"])
    
    # Display the DataFrame
    return df

In [19]:
df0 = generate0(text0)

In [20]:
df = df2 = df2[["County", "District", "County Pop", "District Pop", "County District Pop", "Percent County in District", "Percent District in County"]]

In [21]:
df1 = df1[["County", "District", "County Pop", "District Pop", "County District Pop", "Percent County in District", "Percent District in County"]]

In [22]:
df0 = df0.rename(columns={'Percent of District in County': 'Percent District in County'})

In [23]:
full_df = pd.concat([df0, df1, df2], ignore_index=True)

In [162]:
full_df.to_csv('nc_pop_2016.csv', index=False) 